In [2]:
import nltk
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

nltk.download('punkt_tab')

# dataset = '../data-sets/Zinparen in Engels-Nederlands - 2024-06-07.tsv'
# df = pd.read_csv(
#     dataset,
#     sep='\t',
#     header=None,
#     usecols=[1, 3],
#     names=["ENG", "NLD"],
#     dtype="string",
#     keep_default_na=False)

dataset = '../data-sets/train-00000-of-00001.parquet'
df = pd.read_parquet(dataset)
df = pd.json_normalize(df['translation'])

sos_token = '<sos>'
eos_token = '<eos>'

def tokenize(text, language):
    return [sos_token] + nltk.word_tokenize(text, language=language) + [eos_token]

def tokenize_column(texts, lang):
    return [tokenize(t, lang) for t in texts]

with ThreadPoolExecutor(max_workers=16) as ex:
    eng_future = ex.submit(tokenize_column, df['en'], 'english')
    nld_future = ex.submit(tokenize_column, df['nl'], 'dutch')
    df['ENG_TOKENS'] = eng_future.result()
    df['NLD_TOKENS'] = nld_future.result()

df.drop(columns=['en', 'nl'], axis=1, inplace=True)
df.to_parquet('../data-sets/train-00000-of-00001 - 2025-11-01.parquet', index=False)


print(df)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ocmki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                               ENG_TOKENS  \
0                     [<sos>, It, 's, for, you, ., <eos>]   
1                    [<sos>, You, know, ,, I, ..., <eos>]   
2                            [<sos>, You, knew, ?, <eos>]   
3       [<sos>, -, Do, n't, try, and, talk, anymore, ....   
4               [<sos>, -, I, am, your, friend, ., <eos>]   
...                                                   ...   
999995  [<sos>, It, 's, a, giant, portrait, of, them, ...   
999996  [<sos>, I, do, n't, know, what, to, say, ., <e...   
999997  [<sos>, In, this, case, ,, a, corrective, amou...   
999998  [<sos>, Everything, looks, fine, ,, Jillian, ....   
999999  [<sos>, Well, ,, tell, us, what, you, do, ., <...   

                                               NLD_TOKENS  
0                   [<sos>, Het, is, voor, jou, ., <eos>]  
1                   [<sos>, Weet, je, ,, ik, ...., <eos>]  
2                        [<sos>, Je, wist, het, ?, <eos>]  
3                          